In [3]:
import wandb
import torch
import wandb, pandas as pd, re, os, glob, warnings


In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [ ]:
# ==== USER CONFIG ====
ENTITY  = "yuulia-volkova-algoverse"
PROJECT  = "outlines_linear_probe"

# W&B artifact names 
RES_ARTIFACT   = "res_normalizer.pt:v3"
EMBED_ARTIFACT = "embed_normalizer.pt:v3"   

# Paths for your data + checkpoint
RESIDUALS_PATH = "/workspace/hdd_cache/tensors/llama-3b"
HF_REPO_ID     = "yulia-volkova/llama-3b-outlines-embeddings_new"

# Evaluate the "last 4 chunks"
CHUNK_IDS = list(range(996, 1000))

# Validation split config:
# - If StreamingOutlineDataset is available, this is a global split (90/10).
# - If not, legacy OutlineDataset uses per-chunk split (~900 train / 100 val).
TRAIN_FRAC = 0.9
TRAIN_SIZE_PER_CHUNK = 900  # used only for legacy OutlineDataset (assuming 1000 samples/chunk)

# Probe checkpoint to evaluate (change to your best)
CHECKPOINT   = "results/probes/probe_epoch_1.pt"

# Batch sizes
BATCH_SIZE   = 64
DECODE_BATCH = 4

# Number of qualitative examples to print
NUM_EXAMPLES = 8

# SONAR decoder (you said you have this)
DECODER_MODEL_ID     = "sonar/text-embedding-decoder"  # <-- put the exact decoder you use
DECODE_MAX_NEW_TOKENS = 128
DECODE_TEMPERATURE    = 0.7
DECODE_TOP_P          = 0.95

# Device
import torch
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DEVICE


device(type='cuda')

In [5]:
import wandb

api = wandb.Api()
RUN_PATH = "yuulia-volkova-algoverse/outlines_linear_probe/055rgxqs"  # <-- use the run ID from your screenshot
run = api.run(RUN_PATH)
print("Run name:", run.name, "| Run id:", run.id)

print("\n=== logged_artifacts ===")
logged = list(run.logged_artifacts())
print("count:", len(logged))
for a in logged:
    print(a.name, "| type:", a.type, "| version:", a.version)

print("\n=== used_artifacts ===")
used = list(run.used_artifacts())
print("count:", len(used))
for a in used:
    creator = a.logged_by().name if a.logged_by() else "<unknown>"
    print(a.name, "| type:", a.type, "| version:", a.version, "| created by:", creator)

wandb: Currently logged in as: yuulia-volkova (yuulia-volkova-algoverse) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Run name: probe-local-resids | Run id: 055rgxqs

=== logged_artifacts ===
count: 44
res_normalizer.pt:v3 | type: asset | version: v3
embed_normalizer.pt:v3 | type: asset | version: v3
epoch-001-chunks.txt:v4 | type: metadata | version: v4
checkpoint-epoch-1:v2 | type: model | version: v2
epoch-002-chunks.txt:v1 | type: metadata | version: v1
checkpoint-epoch-2:v0 | type: model | version: v0
epoch-003-chunks.txt:v0 | type: metadata | version: v0
checkpoint-epoch-3:v0 | type: model | version: v0
epoch-004-chunks.txt:v0 | type: metadata | version: v0
checkpoint-epoch-4:v0 | type: model | version: v0
epoch-005-chunks.txt:v0 | type: metadata | version: v0
checkpoint-epoch-5:v0 | type: model | version: v0
epoch-006-chunks.txt:v0 | type: metadata | version: v0
checkpoint-epoch-6:v0 | type: model | version: v0
epoch-007-chunks.txt:v0 | type: metadata | version: v0
checkpoint-epoch-7:v0 | type: model | version: v0
epoch-008-chunks.txt:v0 | type: metadata | version: v0
checkpoint-epoch-8:v0 | ty

In [ ]:
import os, re, glob, warnings, shutil
import pandas as pd
import wandb

ENTITY  = "yuulia-volkova-algoverse"
PROJECT = "outlines_linear_probe"
RUN_ID  = "055rgxqs"
LOCAL_DIR = f"artifacts/checkpoints/{RUN_ID}"
os.makedirs(LOCAL_DIR, exist_ok=True)

api = wandb.Api()
run = api.run(f"{ENTITY}/{PROJECT}/{RUN_ID}")
print("Run:", run.name, "| id:", run.id)

# pick best epoch
hist = run.history(pandas=True)
val_col_candidates = ["epoch/val_loss","val/loss","val_loss","val/mse","epoch/val_mse"]
val_col = next((c for c in val_col_candidates if c in hist.columns and hist[c].notna().any()), None)
h2 = hist[["epoch", val_col]].dropna()
best_epoch = int(h2.loc[h2[val_col].idxmin(), "epoch"])
print(f"Best epoch: {best_epoch}  ({val_col}={float(h2.loc[h2[val_col].idxmin(), val_col]):.6f})")

# find its artifact
chosen_art = None
for art in run.logged_artifacts():
    if art.type == "model" and art.name.startswith("checkpoint-epoch-"):
        m = re.match(r"checkpoint-epoch-(\d+):", art.name)
        if m and int(m.group(1)) == best_epoch:
            chosen_art = art; break
assert chosen_art is not None, "No checkpoint artifact found for that epoch."

print("Chosen artifact:", chosen_art.name)

# build a stable local cache filename (with .pt extension for convenience)
base_name = chosen_art.name.split(":")[0]        # e.g. "checkpoint-epoch-16"
local_ckpt = os.path.join(LOCAL_DIR, base_name + ".pt")

if os.path.exists(local_ckpt):
    print("✅ Using cached checkpoint:", local_ckpt)
    ckpt_path = local_ckpt
else:
    print("⬇️ Downloading from W&B (one time)...")
    ckpt_dir = chosen_art.download(root=LOCAL_DIR)

    manifest_keys = list(chosen_art.manifest.entries.keys())
    if not manifest_keys:
        raise RuntimeError("Artifact has no files in manifest.")
    rel_path = manifest_keys[0]                 # there’s only 1 file per checkpoint
    src_path = os.path.join(ckpt_dir, rel_path)
    if not os.path.exists(src_path):
        # fallback: search deeply
        cand = None
        for root, _, files in os.walk(ckpt_dir):
            for f in files:
                if f.startswith(base_name):      # e.g. "checkpoint-epoch-16"
                    cand = os.path.join(root, f); break
            if cand: break
        if cand is None:
            # last resort: any file
            files = [os.path.join(ckpt_dir, f) for f in os.listdir(ckpt_dir)]
            if not files:
                raise RuntimeError("Downloaded artifact folder is empty.")
            cand = files[0]
        src_path = cand

    # copy to the stable cache path with .pt extension (even if original had none)
    shutil.copy2(src_path, local_ckpt)
    ckpt_path = local_ckpt
    print("Saved to:", ckpt_path)

print("Final checkpoint path:", ckpt_path)

# sanity-load
import torch
state = torch.load(ckpt_path, map_location="cpu")
print("Checkpoint keys:", list(state.keys()))


Run: probe-local-resids | id: 055rgxqs
Best epoch: 17  (epoch/val_loss=0.532388)


wandb: Downloading large artifact 'checkpoint-epoch-17:v0', 2052.02MB. 1 files...


Chosen artifact: checkpoint-epoch-17:v0
⬇️ Downloading from W&B (one time)...


wandb:   1 of 1 files downloaded.  
Done. 00:00:10.3 (199.6MB/s)


Saved to: artifacts/checkpoints/055rgxqs/checkpoint-epoch-17.pt
Final checkpoint path: artifacts/checkpoints/055rgxqs/checkpoint-epoch-17.pt
Checkpoint keys: ['epoch', 'model_state_dict', 'optimizer_state_dict', 'scheduler_state_dict', 'train_loss', 'val_loss']


In [65]:
import train_probe as core  

state = torch.load(ckpt_path, map_location="cpu")

N_LAYERS = 57         
D_MODEL  = 3072       
probe = core.LinearProbe(n_layers=N_LAYERS, d_model=D_MODEL)
probe.load_state_dict(state["model_state_dict"], strict=True)
probe.eval()

print("Loaded epoch (0-based):", state.get("epoch"))


Loaded epoch (0-based): 16


In [ ]:
res_art   = next(a for a in run.logged_artifacts() if a.name.startswith("res_normalizer.pt"))
embed_art = next(a for a in run.logged_artifacts() if a.name.startswith("embed_normalizer.pt"))

res_dir   = res_art.download()
embed_dir = embed_art.download()

from train_probe import Normalizer
res_norm  = Normalizer(**torch.load(os.path.join(res_dir,   "res_normalizer.pt"),   map_location="cpu"))
emb_norm  = Normalizer(**torch.load(os.path.join(embed_dir, "embed_normalizer.pt"), map_location="cpu"))


wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  


In [39]:
import sonar_utils

text2vec, vec2text = sonar_utils.init_sonar()


[SONAR] init on device=cuda, dtype=torch.bfloat16
2025-09-23 19:42:26,410 - Using the cached checkpoint of text_sonar_basic_encoder. Set `force` to `True` to download again.
2025-09-23 19:42:42,819 - Using the cached tokenizer of text_sonar_basic_encoder. Set `force` to `True` to download again.
2025-09-23 19:42:43,202 - Using the cached checkpoint of text_sonar_basic_decoder. Set `force` to `True` to download again.
2025-09-23 19:43:09,874 - Using the cached tokenizer of text_sonar_basic_encoder. Set `force` to `True` to download again.


In [10]:
import os
from pathlib import Path
import einops
from huggingface_hub import hf_hub_download
import torch.nn.functional as F

In [ ]:
RESIDUALS_PATH = "/workspace/hdd_cache/tensors/llama-3b"
EMBEDS_PATH     = "/workspace/ALGOVERSE/yas/yulia/parascopes/src/yulia/outlines/results/llama-3b-outlines-embeddings_new"
HF_REPO_ID     = "yulia-volkova/llama-3b-outlines-embeddings_new"  # only used if you enable fallback
DTYPE          = torch.float32

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [27]:
def eval_chunks(chunk_ids, batch_size=32):
    import torch, torch.nn.functional as F, einops, os
    from pathlib import Path

    global probe, res_norm, emb_norm, RESIDUALS_PATH, EMBEDS_PATH, DTYPE

    # Make sure the probe is on a device and in eval mode
    if next(probe.parameters()).device.type == "cpu" and torch.cuda.is_available():
        probe.to("cuda")
    probe.eval()

    mse_all, cos_all, examples = [], [], []

    for chunk_id in chunk_ids:
        # load residuals
        res_path = Path(RESIDUALS_PATH) / f"res_data_{chunk_id:03d}.pt"
        res_list = torch.load(res_path, map_location="cpu")

        # load embeddings (LOCAL)
        emb_path = Path(EMBEDS_PATH) / f"outlines_{chunk_id:03d}.pt"
        embeds = torch.load(emb_path, map_location="cpu").to(dtype=DTYPE)

        assert len(res_list) == 1000 and len(embeds) == 1000, f"Count mismatch in chunk {chunk_id}"

        # The single source of truth for device is the model's device
        model_dev = next(probe.parameters()).device

        for i in range(0, 1000, batch_size):
            res_batch, emb_batch = [], []
            end = min(i + batch_size, 1000)
            for j in range(i, end):
                r = res_list[j]["res"].to(dtype=DTYPE)               # CPU tensor
                x = einops.rearrange(r[:, :1, :], "l p d -> p l d")  # [1,L,D] on CPU
                x = res_norm.normalize(x)                            # normalize on CPU

                y = emb_norm.normalize(embeds[j])                    # [1024] on CPU
                y = y.unsqueeze(0)                                   # [1,1024] on CPU

                res_batch.append(x)
                emb_batch.append(y)

            # NOW move to the model's device (not a global variable)
            X = torch.cat(res_batch, dim=0).to(model_dev, non_blocking=True)
            Y = torch.cat(emb_batch, dim=0).to(model_dev, non_blocking=True)

            # Safety check (remove after first successful run)
            assert next(probe.parameters()).device == X.device, \
                f"Probe on {next(probe.parameters()).device}, X on {X.device}"

            with torch.no_grad():
                P = probe(X)                             # [B, 1024]
                mse = ((P - Y) ** 2).mean(dim=1)
                cos = F.cosine_similarity(P, Y, dim=1)

            mse_all.extend(mse.detach().cpu().tolist())
            cos_all.extend(cos.detach().cpu().tolist())

            if len(examples) < 12:
                keep = min(3, P.shape[0])
                examples.append({
                    "chunk": chunk_id,
                    "pred": P[:keep].detach().cpu(),
                    "gold": Y[:keep].detach().cpu(),
                })

    mse_t = torch.tensor(mse_all)
    cos_t = torch.tensor(cos_all)
    print(f"Eval on chunks {chunk_ids}:")
    print(f"  MSE  mean={mse_t.mean().item():.4f} | median={mse_t.median().item():.4f}")
    print(f"  COS  mean={cos_t.mean().item():.4f} | median={cos_t.median().item():.4f}")
    return examples


In [28]:

examples = eval_chunks([996], batch_size=32)

Eval on chunks [996]:
  MSE  mean=0.5446 | median=0.5372
  COS  mean=0.6780 | median=0.6804


In [55]:
import torch

def _vec2text_device_dtype():
    """
    Robustly infer the actual device & dtype used by the SONAR decoder weights.
    """
    # Newer sonar has .model; older might differ — try a few options
    for attr in ["model", "_model", "decoder", "_decoder"]:
        m = getattr(vec2text, attr, None)
        if m is not None:
            try:
                p = next(m.parameters())
                return p.device, p.dtype
            except Exception:
                pass

    # Absolute fallback: try attributes or default to CUDA bf16 if available
    dev = getattr(vec2text, "device", torch.device("cuda" if torch.cuda.is_available() else "cpu"))
    dt  = getattr(vec2text, "dtype", torch.bfloat16 if dev.type == "cuda" else torch.float32)
    return dev, dt


def decode_embeddings(tensors, target_lang="eng_Latn"):
    """
    tensors: list[1D torch.Tensor] or a single 2D torch.Tensor [B, 1024]
    Cast inputs to the *actual* decoder model’s device & dtype before predict().
    """
    # normalize to list of 1D tensors
    if isinstance(tensors, torch.Tensor):
        if tensors.ndim == 1:
            batch = [tensors]
        elif tensors.ndim == 2:
            batch = [tensors[i] for i in range(tensors.size(0))]
        else:
            raise ValueError("Expected 1D or 2D tensor for embeddings.")
    else:
        batch = list(tensors)

    dec_device, dec_dtype = _vec2text_device_dtype()

    # IMPORTANT: single-step cast to both device & dtype
    batch = [t.detach().to(device=dec_device, dtype=dec_dtype) for t in batch]

    # SONAR expects a list of tensors
    return vec2text.predict(batch, target_lang=target_lang)


In [57]:
# This should run without "Float vs BFloat16" now
fake = torch.randn(1024)
print(decode_embeddings([fake, fake]))


['each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each each eac

In [62]:
import torch, einops
import torch.nn.functional as F
from pathlib import Path
import pandas as pd


DEVICE = next(probe.parameters()).device

def emb_restore(z_norm: torch.Tensor) -> torch.Tensor:
    return z_norm * (emb_norm.std + 1e-6) + emb_norm.mean

import pandas as pd

def eval_and_decode_chunks(chunks, per_chunk=10, batch_size=32, csv_path="decoded_outputs.csv"):
    probe.eval().to(DEVICE)

    all_mse, all_cos = [], []
    rows = []  # <-- collect rows for CSV

    for chunk_id in chunks:
        # load local residuals + embeddings
        res_path = Path(RESIDUALS_PATH) / f"res_data_{chunk_id:03d}.pt"
        emb_path = Path(EMBEDS_PATH)    / f"outlines_{chunk_id:03d}.pt"
        res_list = torch.load(res_path, map_location="cpu")
        embeds   = torch.load(emb_path,  map_location="cpu").to(dtype=DTYPE)

        assert len(res_list) == 1000 and len(embeds) == 1000, f"Count mismatch in chunk {chunk_id}"

        # --- evaluate whole chunk (normalized metrics) ---
        for i in range(0, 1000, batch_size):
            res_batch, emb_batch = [], []
            end = min(i + batch_size, 1000)
            for j in range(i, end):
                r = res_list[j]["res"].to(dtype=DTYPE)                 # [L, P, D]
                x = einops.rearrange(r[:, :1, :], "l p d -> p l d")    # [1, L, D]
                x = res_norm.normalize(x)                              # CPU
                y = emb_norm.normalize(embeds[j]).unsqueeze(0)         # [1, 1024]
                res_batch.append(x); emb_batch.append(y)

            X = torch.cat(res_batch, 0).to(DEVICE, non_blocking=True)  # [B, L, D]
            Y = torch.cat(emb_batch, 0).to(DEVICE, non_blocking=True)  # [B, 1024]
            with torch.no_grad():
                P = probe(X)
                all_mse.extend(((P - Y) ** 2).mean(dim=1).cpu().tolist())
                all_cos.extend(F.cosine_similarity(P, Y, dim=1).cpu().tolist())

        # --- decode ONLY `per_chunk` samples (evenly spaced across the 1000) ---
        if per_chunk <= 0:
            continue
        step = max(1000 // per_chunk, 1)
        decode_idxs = [min(k * step, 999) for k in range(per_chunk)]

        print(f"\n=== Chunk {chunk_id} : decoding {len(decode_idxs)} samples ===")
        for idx in decode_idxs:
            r = res_list[idx]["res"].to(dtype=DTYPE)
            x = einops.rearrange(r[:, :1, :], "l p d -> p l d")  # [1, L, D]
            x_n = res_norm.normalize(x)
            y_n = emb_norm.normalize(embeds[idx]).unsqueeze(0)   # [1, 1024]

            with torch.no_grad():
                y_hat_n = probe(x_n.to(DEVICE)).squeeze(0)

            mse = F.mse_loss(y_hat_n, y_n.to(DEVICE)).item()
            cos = F.cosine_similarity(y_hat_n.unsqueeze(0), y_n.to(DEVICE), dim=1).item()
            print(f"[{idx:04d}] MSE={mse:.6f} | COS={cos:.4f}")

            # restore embeddings for decoding
            y_hat_rest = emb_restore(y_hat_n)
            y_gt_rest  = emb_restore(y_n.squeeze(0))

            decoded = decode_embeddings([y_gt_rest, y_hat_rest])

            print("Original decoded:",  decoded[0])
            print("Predicted decoded:", decoded[1])
            print("-" * 80)

            # add row to CSV list
            rows.append({
                "chunk": chunk_id,
                "index": idx,
                "mse": mse,
                "cosine": cos,
                "decoded_original": decoded[0],
                "decoded_predicted": decoded[1],
            })

    # overall metrics
    mse_t = torch.tensor(all_mse); cos_t = torch.tensor(all_cos)
    print("\n=== Overall metrics across chunks", chunks, "===")
    print(f"MSE  mean={mse_t.mean().item():.4f} | median={mse_t.median().item():.4f}")
    print(f"COS  mean={cos_t.mean().item():.4f} | median={cos_t.median().item():.4f}")

    # save CSV
    if rows:
        df = pd.DataFrame(rows)
        df.to_csv(csv_path, index=False)
        print(f"\n✅ Saved decoded outputs to {csv_path}")
        return df
    else:
        print("No decoded samples collected.")
        return None


In [59]:
fake = torch.randn(1024)
decoded = decode_embeddings([fake, fake])
print(decoded)


['here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here here her

In [64]:
# run:
eval_and_decode_chunks([996, 997, 998, 999], per_chunk=10, batch_size=32)


=== Chunk 996 : decoding 10 samples ===
[0000] MSE=0.632150 | COS=0.6562


/tmp/ipykernel_513527/1806812549.py:63: UserWarning: Using a target size (torch.Size([1, 1024])) that is different to the input size (torch.Size([1024])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  mse = F.mse_loss(y_hat_n, y_n.to(DEVICE)).item()


Original decoded: Summary: 1.History map of the Helsinki University Library - reflects Finnish history and politics 2.Early Swedish identity and national identity - Influenced by Swedish and Russian history through the formation of the map 3.Use of national identity and sovereignty to demonstrate autocracy 4.Impacts on historical events - Finnish war and national revival 5.Philosophical heritage and the preservation of Finland's national identity - Essential to understanding the significance of Finnish maps
Predicted decoded: Description: 1. Baseline of the Maltese history - Cultural exhibitions 2. History of the Netherlands - Contextual significance 3. Cultural exhibitions - History of the Netherlands - History of the Netherlands - History of the Netherlands - History of the Netherlands - History of the Netherlands - History of the Netherlands
--------------------------------------------------------------------------------
[0100] MSE=0.781245 | COS=0.5635
Original decoded: Summary: - 

,chunk,index,mse,cosine,decoded_original,decoded_predicted
0,996,0,0.632150,0.656229,Summary: 1.History map of the Helsinki Univers...,Description: 1. Baseline of the Maltese histor...
1,996,100,0.781245,0.563521,Summary: - Establishment of Little League Base...,Summary: 1. Founding of National Soccer League...
2,996,200,0.685692,0.614616,Luteinizing hormone (LH) function - Regulates ...,Description: 1.Lymphocytic Axis - Definition o...
3,996,300,0.602757,0.630434,Description 3: 3-D technology used in ancient ...,Summary: 1.Discovery of 3D spacecraft - Innova...
4,996,400,0.504234,0.666083,Explanation: 1. The economic impact of tobacco...,Summary: 1. The legacy of tobacco - American a...
5,996,500,0.700365,0.625722,Summary: 1.Spanish settlers set up dairy farms...,Summary: 1.History of dairy farming - Upper Vi...
6,996,600,0.638288,0.632294,Summary: 1.Pediatric Cancer Genome Research Pr...,Summary: 1. Progress of St. John's Wort Diseas...
7,996,700,0.542620,0.717413,Description: Lechordus habitat - East Australi...,Description: 1.Larpshoes - List of species of ...
8,996,800,0.576993,0.663050,Summary: Hooverville - Simulated Voluntary Edu...,Summary: 1. Programme - Explore the Watershed ...
9,996,900,0.430678,0.744383,Schedule 1: Protects ecosystems as aquifers - ...,Summary: 1.Pleasure of lakes - Ecological life...
